In [ ]:
# PLAN
  # import libraries
  
  # define address array
  
  # define filename queue
  
  # define image reader
  
  # execute session
    # define coordinator object to manage threads
  
    # define threads using start_queue_runners method
      # one thread processes one image
      # queue runner produces threads and sets them up in parallel
      # pass in coordinator object to manage the threads
    
    # convert images to list of arrays
      # create empty list
      # read one file from the queue
      # decode
      # resize
      # set shape
      # perform transformations
      # sess.run to convert to numpy array
      # tf.stack to convert to tensor
      # append tensor to image list
    
    # stop processing
    
    # join threads
    
    # write image summary
    
  # view in tensorboard
  

In [1]:
# IMPORT LIBRARIES
import tensorflow as tf
import google.datalab.ml as ml

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# DEFINE ADDRESS ARRAY CONTAINING IMAGE ADDRESSES
original_image_list = ["./images/cat.2.jpg", 
              "./images/cat.3.jpg",
              "./images/cat.4.jpg",
              "./images/cat.5.jpg",
              "./images/cat.6.jpg",
              "./images/dog.1.jpg",
              "./images/dog.2.jpg",
              "./images/dog.3.jpg",
              "./images/dog.4.jpg",
              "./images/dog.5.jpg"
             ]

In [3]:
# DEFINE FILENAME QUEUE USING ADDRESS ARRAY
filename_queue = tf.train.string_input_producer(original_image_list) # we will produce multiple threads to read from this queue of images

In [4]:
# DEFINE IMAGE READER
image_reader = tf.WholeFileReader() # allows us to read the entire file in one go

In [5]:
# EXECUTE SESSION
with tf.Session() as sess:
    
    # create queue runner object to coordinate the loading of image files
    
    coord = tf.train.Coordinator() # class which allows you to coordinate multiple threads; a method call on the coordinator can be used to stop processing on all your threads
    
    threads = tf.train.start_queue_runners(sess = sess, coord = coord) # queue runner allows you to process elements in a queue in parallel; purpose of threads is to read from the queue of images; each image will be processed on a different thread
  
  
  
    # convert images to list of tensors
    
    image_list = []
    
    for i in range(len(original_image_list)):
        
        # read one file from the queue (using the image reader you created)
        _, image_file = image_reader.read(filename_queue) # image_reader.read returns a tuple (filename, filecontent); we will ignore the filename
    
    
        # decode
        image = tf.image.decode_jpeg(image_file)
    
    
        # resize
        image = tf.image.resize_images(image, [224,224])
    
    
        # set shape
        image.set_shape((224,224,3))
    
    
        # perform transformations
    
            # flip upside down
        image = tf.image.flip_up_down(image)
    
    
            # crop away a border
        image = tf.image.central_crop(image, central_fraction=0.5)
    
    
        # sess.run to convert to numpy array
        image_array = sess.run(image)
        print image_array.shape
    
    
        # tf.stack to convert to tensor
        image_tensor = tf.stack(image_array)
        print image_tensor
        print " "
    
    
        # append tensor to image list
        image_list.append(image_tensor)
    
       
    # stop processing
    coord.request_stop()


    # join threads
    coord.join(threads)
    
    
    # stack tensors: convert all tensors into a single tensor with a 4th dimension
    images_tensor = tf.stack(image_list)
    print images_tensor
    


    # write image summary
    summary_writer = tf.summary.FileWriter('./resizedimages2', graph=sess.graph)
    
    summary = sess.run(tf.summary.image("images", images_tensor)) # write a summary of your images
    
    summary_writer.add_summary(summary)
    
    summary_writer.close()
  
  

(112, 112, 3)
Tensor("stack:0", shape=(112, 112, 3), dtype=float32)
 
(112, 112, 3)
Tensor("stack_1:0", shape=(112, 112, 3), dtype=float32)
 
(112, 112, 3)
Tensor("stack_2:0", shape=(112, 112, 3), dtype=float32)
 
(112, 112, 3)
Tensor("stack_3:0", shape=(112, 112, 3), dtype=float32)
 
(112, 112, 3)
Tensor("stack_4:0", shape=(112, 112, 3), dtype=float32)
 
(112, 112, 3)
Tensor("stack_5:0", shape=(112, 112, 3), dtype=float32)
 
(112, 112, 3)
Tensor("stack_6:0", shape=(112, 112, 3), dtype=float32)
 
(112, 112, 3)
Tensor("stack_7:0", shape=(112, 112, 3), dtype=float32)
 
(112, 112, 3)
Tensor("stack_8:0", shape=(112, 112, 3), dtype=float32)
 
(112, 112, 3)
Tensor("stack_9:0", shape=(112, 112, 3), dtype=float32)
 
Tensor("stack_10:0", shape=(10, 112, 112, 3), dtype=float32)


In [6]:
# VIEW IN TENSORBOARD
tboard_001 = ml.TensorBoard.start('./resizedimages2')

TensorBoard was started successfully with pid 5702. Click here to access it.

In [7]:
ml.TensorBoard.stop(tboard_001)